In [1]:

import torch
print(torch.__version__)

1.7.1


In [2]:
from __future__ import print_function

import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision import models
import numpy
import dataset
import dataset_test
import random
from utils import *
from VGG_gru import FERANet
# from tensorboardX import SummaryWriter
from torch.utils.tensorboard import SummaryWriter
import argparse
import subprocess
import math
import csv
import pandas as pd
import copy


## Loading Model

In [3]:
# model can be downloaded here: https://drive.google.com/drive/folders/1f17xgwvGaUpgXYBssocUNXDBgga-b3qp
preInitial = False
if preInitial == True:
	model = FERANet()
	model = Initial(model)
else:
	model = FERANet()
	model = VGG_Initial(model)

learning_rate = 0.0001
optimizer = optim.SGD(model.parameters(), lr=learning_rate , momentum=0.9, weight_decay= 0.00005)
#https://discuss.pytorch.org/t/problem-about-k-fold-validation/32334/2    
init_state = copy.deepcopy(model.state_dict())
init_state_opt = copy.deepcopy(optimizer.state_dict())


start loading VGG-Face model...
finish loading VGG-Face model!


In [4]:
processed_batches = 0
num_workers   = 0
torch.manual_seed(1)
torch.cuda.manual_seed(1)
random.seed(1)
torch.backends.cudnn.benchmark = True

kwargs = {'num_workers': num_workers, 'pin_memory': True}



### MODEL PARAMETERS:

In [5]:
batch_size = 1
# learning_rate = 0.0001
epochs = 20 
length = 32

use_Tensorboard = False

### TO CHANGE:

In [6]:
###### TO CHANGE #######
# EXP_TYPE = 'Regression/'
EXP_TYPE = 'Classification/'

#### Change loss function in UTILS.py

metric = AccumulatedAccuracyMetric()

### TO KEEP:

In [7]:
def train(epoch,optimizer):
      
	train_loader = torch.utils.data.DataLoader(
		dataset.listDataset(trainlist,length = length,
					   shuffle=True,
					   train=True,
					   dataset = dataset),
		batch_size=batch_size, shuffle=False, **kwargs)


	for param_group in optimizer.param_groups:
		train_learning_rate = float(param_group['lr'])

	logging('epoch %d, processed %d samples, lr %f' % (epoch, epoch * len(train_loader.dataset), train_learning_rate))

	running_loss = 0.0

	model.train()

	for batch_idx, (data, label) in enumerate(train_loader):


		data = data.squeeze(0)
		data = Variable(data).cuda()

		label = Variable(label.long()).cuda()
		label = label.squeeze(1)
        
		optimizer.zero_grad()

		output = model(data)

		loss = loss_function(output, label)

		running_loss += loss.data

		loss.backward()

		optimizer.step()

	if epoch %1 == 0:
		logging('Train Loss:{:.6f}'.format(running_loss))
        
	return running_loss        

In [8]:
def eval(epoch,metric):

	model.eval()
    
	test_loader = torch.utils.data.DataLoader(
		dataset.listDataset(vallist,length = length,       
					shuffle=False,
					train=False,
					dataset = dataset),
					batch_size=1, shuffle=False, **kwargs)

	for batch_idx, (data, label) in enumerate(test_loader):

		data = data.squeeze(0)
		Batch,T,C,H,W = data.size()

		data = Variable(data,volatile=True).cuda()

		label = Variable(label.long(),volatile=True).cuda()
		label = label.squeeze(1)
		output = []
        
		for batch_index in range(Batch):
# 			print(batch_index) 
# 			with torch.no_grad():
			output_feature = model(data[batch_index])
			output.append(output_feature)
		output = torch.mean(torch.cat(output), 0, keepdim=True) 
# 		print("Dua's output is: ", output)
# 		with torch.no_grad():
		metric(output, label) 
		eval_loss = metric.value()
	if epoch %1 == 0:
		logging("Eval Loss: %f" % (eval_loss))
    
	return eval_loss

In [9]:
def test(epoch,metric):
    
	model.eval()
	result = []    
	test_loader = torch.utils.data.DataLoader(
		dataset_test.listTESTDataset(testlist,length = length,
# 		dataset.listDataset(vallist,length = length,        
					shuffle=False,
# 					train=False,
					dataset = dataset),
					batch_size=1, shuffle=False, **kwargs)
    
	for batch_idx, (data, label, participant) in enumerate(test_loader):

		data = data.squeeze(0)

		Batch,T,C,H,W = data.size()
		data = Variable(data,volatile=True).cuda()

		label = Variable(label.long(),volatile=True).cuda()
		label = label.squeeze(1)
# 		print('Participant name:',participant)
# 		print('Participant label:',int(label))
		output = []
		for batch_index in range(Batch):           
			output_feature = model(data[batch_index])
			output.append(output_feature)
		output = torch.mean(torch.cat(output), 0, keepdim=True)      
# 		print("Participant output is: ", output)
		metric(output, label) 
		test_loss = metric.value()
# 		print("Participant loss is: ", test_loss)
		result.append([participant, int(label), output, test_loss])
	return result

In [10]:
COMB = ['BDI','AVEC','AVEC-BDI','AVEC-TEST','BDI-TEST']
NUM = ['1','2','3','4','5']
#Text file of image path for each of the above combinations
textMAIN = '/timo/datasets/Dua/GRU-test/Data-Combinations/'
textPATH = textMAIN+EXP_TYPE
backupdir     = '/timo/datasets/Dua/GRU-test/Saved-Models'
results_PATH = '/timo/datasets/Dua/GRU-test/Results/'
#Datasets Path (participants Path) AVEC + BDI
# path_d= '/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/dataset.py'
#Where to save trained models Path

In [11]:
# if os.path.isfile != True:
# 	subprocess.call(["python", path_d])

## Training+Evaluating 

In [12]:
trn_tensor_writer = SummaryWriter()
val_tensor_writer = SummaryWriter()
for C in COMB:
    for N in NUM:
        model.load_state_dict(init_state)
        model = model.cuda()
        optimizer.load_state_dict(init_state_opt)
        train_val_loss = []
        trainlist=textPATH+C+'train'+N+'.txt'
        vallist=textPATH+C+'val'+N+'.txt'
        for epoch in range(1, epochs+1): 
            train_loss = train(epoch,optimizer)
            with torch.no_grad():
                eval_loss = eval(epoch,metric)
            train_val_loss.append([int(train_loss), int(eval_loss)]) 
            trn_tensor_writer.add_scalar("Loss/train", train_loss, epoch)
            val_tensor_writer.add_scalar("Loss/val", eval_loss, epoch)
        with open(results_PATH+C+'train_val'+N+'_loss.csv', 'w') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerows(train_val_loss)
        trn_tensor_writer.flush()
        val_tensor_writer.flush()
        csvFile.close()
        torch.save(model.state_dict(),'%s/model_%s%s.pkl' % (backupdir,C,N))
trn_tensor_writer.close()
val_tensor_writer.close()

        #add saving train +eval loss

2021-03-02 07:49:15 epoch 1, processed 36 samples, lr 0.000100
2021-03-02 07:49:26 Train Loss:26.669254


/home/duaa/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/duaa/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


2021-03-02 07:52:14 Eval Loss: 7.546226
2021-03-02 07:52:14 epoch 2, processed 72 samples, lr 0.000100
2021-03-02 07:52:25 Train Loss:23.759930
2021-03-02 07:55:11 Eval Loss: 14.874059
2021-03-02 07:55:11 epoch 3, processed 108 samples, lr 0.000100
2021-03-02 07:55:22 Train Loss:21.099333
2021-03-02 07:58:06 Eval Loss: 21.844591
2021-03-02 07:58:06 epoch 4, processed 144 samples, lr 0.000100
2021-03-02 07:58:17 Train Loss:20.209955
2021-03-02 08:01:00 Eval Loss: 28.441851
2021-03-02 08:01:00 epoch 5, processed 180 samples, lr 0.000100
2021-03-02 08:01:10 Train Loss:16.165459
2021-03-02 08:03:52 Eval Loss: 37.737408
2021-03-02 08:03:52 epoch 6, processed 216 samples, lr 0.000100
2021-03-02 08:04:02 Train Loss:16.196770
2021-03-02 08:06:44 Eval Loss: 44.149010
2021-03-02 08:06:44 epoch 7, processed 252 samples, lr 0.000100
2021-03-02 08:06:55 Train Loss:12.583697
2021-03-02 08:09:36 Eval Loss: 53.645378
2021-03-02 08:09:36 epoch 8, processed 288 samples, lr 0.000100
2021-03-02 08:09:47 T

KeyboardInterrupt: 

## Testing Results

In [ ]:
for C in COMB:
    for N in NUM:
        testlist=textPATH+C+'test'+N+'.txt'
        model.load_state_dict(torch.load('%s/model_%s%s.pkl' % (backupdir,C,N)))
        with torch.no_grad():
            test_accuary = test(epoch,metric)
        with open(results_PATH+C+'test'+N+'Results.csv', 'w') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerows(test_accuary)
        csvFile.close()

In [ ]:
# data = pd.read_csv('/timo/datasets/Dua/GRU-test/Results/test.csv')
# data